<a href="https://colab.research.google.com/github/NunezKant/COVID19_MEX_MASTER/blob/master/COVID_MEX_MASTER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
!pip install arviz
import numpy as np
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns
import arviz as az
import pymc3 as pm
import altair as alt
import plotly.express as px
import requests
import io

sns.set_context('talk')
plt.style.use('seaborn-whitegrid')

In [0]:
#INITS

dias_evaluacion = 120 #Mejora en convergencia de ecuaciones diferenciales
dt=.01
periodo_evaluacion = np.arange(0,dias_evaluacion+dt,dt)
alpha = 0.2
gamma = 0.5

#Escenarios
def Base():
    Ro = 3.5
    gamma = 0.5
    Beta = Ro*gamma
    return Beta
 
def Pesimista():
    Ro = 4.5
    gamma = 0.5
    Beta = Ro*gamma
    return Beta
 
def Optimista():
    Ro = 1.5
    gamma = 0.5
    Beta = Ro*gamma
    return Beta

def Fuerte():
    Ro = 3.5*(0.5)
    gamma = 0.5
    Beta = Ro*gamma
    return Beta

def Moderada():
    Ro = 3.5 - (3.5*(0.25))
    gamma = 0.5
    Beta = Ro*gamma
    return Beta
 
switcher = {
   "Base": Base,
   "Pesimista": Pesimista,
   "Optimista": Optimista,
   "Fuerte": Fuerte,
   "Moderada": Moderada
    }
 
def get_Beta(argument):
# Get the function from switcher dictionary
  func = switcher.get(argument, "nothing")
# Execute the function
  return func()

rho = 0.82
kappa = .016
parametros = alpha, gamma, rho , kappa
Population = 8000000 
I_o = 32 / Population  # Tenemos 32 casos
E_o = (32*4)/ Population # Asumimos 4 expuestos por caso
A_o =  4 / Population
D_o = 0 # No Muertos
S_o = (1) - (E_o+I_o+A_o+D_o) # El resto somos suceptibles
R_o = 0 # NO hay ningun recuperado


Condiciones_Iniciales = S_o,E_o,A_o,I_o,R_o,D_o

In [0]:
def SEAIR_D(IC, pmtrs, t, intervencion = "Base", inicio_intervecion=1, duracion_intervencion=120):
  """
  Esta parte de código es la que implementa las ecuaciones diferenciales.

  Se resuelven por medio de integración númerica de Euler

  Inputs: Condiciones iniciales, Parámetros, Tiempo, Dia de inicio de intervención, Periodo de la intervencion

  Outputs: Suceptibles, Expuestos, Asintomaticos/No Reportados, Infectados, Recuperados, Muertos, Tipo de intervencion

  Parametros: 
              Alpha: Inverso del periodo de incubacion (0.2)
              Beta: Variable (Base: 1.8)
              Gamma: Tiempo Latente (0.5) 
              Ro: Variable (Base: 3.5)
              rho: Proporcion de asintomaticos (.82)
              kappa: Tasa de mortalidad (.016)
  """
  
  So,Eo,Ao,Io,Ro,Do = IC
  S,E,A,I,R,D = [So],[Eo],[Eo],[Io],[Ro],[Do]
  alpha,gamma,rho,kappa = pmtrs
  dt = t[1]-t[0]
  for i in t[1:]:
    if (i >= inicio_intervecion and i <=duracion_intervencion):
      beta = get_Beta(intervencion)
    else:
      beta = get_Beta("Base")


    St = S[-1] - (beta*S[-1]*I[-1])*dt
    Et = E[-1] + (beta*S[-1]*I[-1] - alpha*E[-1])*dt
    At = A[-1] + (1-rho)*(alpha*E[-1] - gamma*A[-1])*dt
    It = I[-1] + rho*(alpha*E[-1] - gamma*I[-1])*dt
    Dt = kappa*I[-1]
    Rt = 1 - (S[-1]+E[-1]+A[-1]+I[-1]-D[-1])
    
    S.append(St)
    E.append(Et)
    A.append(At)
    I.append(It)
    R.append(Rt)
    D.append(Dt)
  return S,E,A,I,R,D, intervencion

Ejemplo de uso:

Intervencion Fuerte constante (Todo el periodo de evaluación)

In [4]:
S,E,A,I,R,D, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Fuerte",1,120)
E_a = np.array(E)*Population
I_a = np.array(I)*Population
A_a = np.array(A)*Population
D_a = np.array(D)*Population
Clase = np.array(["Expuestos"]*np.array(E).shape[0] + ["Infectados"]*np.array(E).shape[0]+ 
                  ["Asintomaticos o No reportados"]*np.array(E).shape[0] + ["Muertes"]*np.array(E).shape[0])
Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a,A_a,D_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase',color_discrete_sequence=["green", "red", "goldenrod", "blue"], template = "ggplot2")

fig.update_layout(
    title=f"Predicción del modelo SEAIR de la evolución de COVID-19 en Jalisco, Escenario: {intervencion}",
    
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

# Casos graves en todos los escenarios

In [5]:
_,_,_,I_base,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Base",1,120)
_,_,_,I_opt,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Optimista",1,120)
_,_,_,I_pes,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Pesimista",1,120)
_,_,_,I_Fuerte,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Fuerte",1,120)
_,_,_,I_Moderada,_,_, intervencion = SEAIR_D(Condiciones_Iniciales,parametros,periodo_evaluacion,"Moderada",1,120)


I_base = np.array(I_base)*Population * .02
I_opt = np.array(I_opt)*Population * .02
I_pes = np.array(I_pes)*Population * .02
I_Fuerte = np.array(I_Fuerte)*Population * .02
I_Moderada = np.array(I_Moderada)*Population * .02
Clase = np.array(["Graves - Pesimista"]*np.array(I_base).shape[0] + ["Graves - Base"]*np.array(I_base).shape[0]+ 
                  ["Graves - Intervencion Moderada"]*np.array(I_base).shape[0] + 
                 ["Graves - Interveción Fuerte"]*np.array(I_base).shape[0]+ ["Graves - Optimista"]*np.array(I_base).shape[0])

Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([I_pes,I_base,I_Moderada,I_Fuerte,I_opt]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase', color_discrete_sequence=px.colors.sequential.RdBu, template = "ggplot2")

fig.update_layout(
    title="Predicción del casos graves de COVID-19 en Jalisco, en distintos Escenarios",
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

# Agrego un loader de las series de tiempo epidemiológicas por Estado, puede ser de ayuda próximamente

In [0]:
def TimeSeriesLoader(url):
  df = pd.read_csv(url)

  df['Inicio'] = pd.to_datetime(df['Inicio'])
  df = (df.drop(["Procedencia", "Llegada","Inicio","Caso","Sexo","Edad","Confirmación"], axis=1)
          .dropna()
          .rename(columns = {"Reporte":"Dia"})
          .sort_values(by = ["Dia"])
          .reset_index(drop = True)
        )
  df["Casos_Acumulados"] = np.nan
  Estados = pd.DataFrame({
    "Estado":[],
    "Dia":[],
    "Casos_Acumulados":[]
  })

  for estado in df.Estado.unique():
      df.loc[(df.Estado == estado),"Casos_Acumulados"] =  np.arange(1,len(df.loc[(df.Estado == estado),"Casos_Acumulados"])+1,1)
      tmp = df.loc[df.Estado == estado].drop_duplicates("Dia", keep = 'last').reset_index(drop = True)
      Estados = pd.concat([Estados,tmp])
  return Estados

In [7]:
Estados = TimeSeriesLoader("https://gist.githubusercontent.com/said3427/18f39eab460766a17ea4802607dd5522/raw") #Leer Datos
Estados.head()

,Estado,Dia,Casos_Acumulados
0,Ciudad De México,2020-02-27,1.0
1,Ciudad De México,2020-02-28,2.0
2,Ciudad De México,2020-03-07,3.0
3,Ciudad De México,2020-03-11,5.0
4,Ciudad De México,2020-03-13,11.0


In [8]:
import plotly.express as px
fig = px.line(Estados,x="Dia",y="Casos_Acumulados",color="Estado", template = "ggplot2",).update_layout(
    title="Casos acumulados de COVID-19 en México",
    xaxis_title="Días",
    yaxis_title="Casos Totales Acumulados",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]
fig.show()